In [1]:
import sys
from sklearn import svm
import adversaries.coordinate_greedy
import adversaries.simple_optimize
from data_reader import input, output, operations
from classifier_wrapper import Classifier
from evasion_metrics import EvasionMetrics

#-----------------------------Library Demo---------------------------------
# Demonstrate framework testing by simulating an SVM classifier and its  
# resulting classification accuracy after an attack from the simple_optimize.py adversary.

In [2]:
learning_model = svm.SVC(probability=True, kernel='linear')

In [3]:
clf = Classifier(learning_model, "100_instance_debug")

In [4]:
clf.train()

In [5]:
adv = adversaries.simple_optimize.Adversary()

In [6]:
instances = input.load_instances(["100_instance_debug", 'test'])